In [1]:
import numpy as np
import cv2
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
os.chdir("/gdrive/MyDrive/ISBI_Dataset")
X_ids = next(os.walk('train'))[2]
Y_ids = next(os.walk('label'))[2]
print(len(X_ids),len(Y_ids))
X_ids.sort()
Y_ids.sort()

In [ ]:
IMG_CHANNELS, IMG_WIDTH, IMG_HEIGHT = 3, 512, 512

In [ ]:
def vertical_symmetry(img_input):
    return np.array(img_input)[::-1,:,:]
    
def horizental_symmetry(img_input):
    return np.array(img_input)[:,::-1,:]

def vertical_symmetry_2D(img_input):
    return np.array(img_input)[::-1,:]
    
def horizental_symmetry_2D(img_input):
    return np.array(img_input)[:,::-1]

def cropping(img_input):
    return img_input.crop((0,0,img_input.width/2,img_input.height/2)),img_input.crop((0,img_input.height/2-1,img_input.width/2,img_input.height-1)),img_input.crop((img_input.width/2-1,0,img_input.width-1,img_input.height/2-1)),img_input.crop((img_input.width/2-1,img_input.height/2-1,img_input.width-1,img_input.height-1))

In [ ]:
X_train = np.zeros((len(X_ids)*7, 256, 256, 3), dtype=np.float32)
Y_train = np.zeros((len(Y_ids)*7, 256, 256, 1), dtype=np.bool)
X_original=np.zeros((len(X_ids),512,512,3),dtype=np.float32)
Y_original=np.zeros((len(Y_ids),512,512,1),dtype=np.float32)

n=0
m=0
for id_ in (X_ids):
    image = tf.keras.preprocessing.image.load_img(f'/gdrive/MyDrive/ISBI_Dataset/train/{id_}', target_size=(IMG_HEIGHT, IMG_WIDTH))
    # print(n,id_)
    input_arr = tf.keras.preprocessing.image.img_to_array(image)[90:450,150:406]
    image = tf.keras.preprocessing.image.array_to_img(input_arr, ).resize((256, 256))
    original_image=tf.keras.preprocessing.image.array_to_img(input_arr, ).resize((512, 512))
    X_train[n] = np.array(image)
    X_original[m]=np.array(original_image)
    img1,img2,img3,img4=cropping(tf.keras.preprocessing.image.array_to_img(X_original[m]))
    n+=1
    X_train[n]=np.array(img1.resize((256, 256)))
    n+=1
    X_train[n]=np.array(img2.resize((256, 256)))
    n+=1
    X_train[n]=np.array(img3.resize((256, 256)))
    n+=1
    X_train[n]=np.array(img4.resize((256, 256)))
    n+=1
    X_train[n]=np.array(tf.keras.preprocessing.image.array_to_img(vertical_symmetry(tf.keras.preprocessing.image.array_to_img(X_train[temp])),).resize((256, 256)))
    n+=1
    X_train[n]=np.array(tf.keras.preprocessing.image.array_to_img(horizental_symmetry(tf.keras.preprocessing.image.array_to_img(X_train[temp])),).resize((256, 256)))
    n+=1
    m+=1

n=0
m=0
for  id_ in (Y_ids):
    image = tf.keras.preprocessing.image.load_img(f'/gdrive/MyDrive/ISBI_Dataset/label/{id_}', 
                                                  target_size=(IMG_HEIGHT, IMG_WIDTH), color_mode="grayscale")

    print(n,id_)
    input_arr = tf.keras.preprocessing.image.img_to_array(image)[90:450,150:406]
    image = tf.keras.preprocessing.image.array_to_img(input_arr, ).resize((256, 256))
    image = tf.keras.preprocessing.image.array_to_img(input_arr, ).resize((256, 256))
    original_image=tf.keras.preprocessing.image.array_to_img(input_arr, ).resize((512, 512))
    Y_train[n] = np.array(image)[:, :, np.newaxis]
    Y_original[m]=np.array(original_image)[:, :, np.newaxis]
    
    img1,img2,img3,img4=cropping(tf.keras.preprocessing.image.array_to_img(Y_original[m]))
    n+=1
    Y_train[n]=np.array(img1.resize((256, 256)))[:, :, np.newaxis]
    n+=1
    Y_train[n]=np.array(img2.resize((256, 256)))[:, :, np.newaxis]
    n+=1
    Y_train[n]=np.array(img3.resize((256, 256)))[:, :, np.newaxis]
    n+=1
    Y_train[n]=np.array(img4.resize((256, 256)))[:, :, np.newaxis]
    n+=1
    Y_train[n]=np.array(vertical_symmetry_2D(tf.keras.preprocessing.image.array_to_img(Y_train[temp])),)[:, :, np.newaxis]
    n+=1
    Y_train[n]=np.array(horizental_symmetry_2D(tf.keras.preprocessing.image.array_to_img(Y_train[temp])),)[:, :, np.newaxis]
    n+=1
    m+=1

In [ ]:
ax1 = plt.subplot(121)
ax1.imshow(tf.keras.preprocessing.image.array_to_img(X_train[29]))
ax2=plt.subplot(122)
ax2.imshow(tf.keras.preprocessing.image.array_to_img(Y_train[29]))
plt.show()

In [3]:
inputs=tf.keras.layers.Input((256,256,3),name="Inputs")
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)


conv1=tf.keras.layers.ZeroPadding2D((1,1),input_shape=(3,256,256))(s)
conv1=tf.keras.layers.Convolution2D(64,3,3,activation='relu')(conv1)
conv1=tf.keras.layers.ZeroPadding3D((1,1))(conv1)
conv1=tf.keras.layers.Convolution2D(64,3,3,activation='relu')(conv1)
conv1=tf.keras.layers.MaxPooling2D((2,2),strides=(2,2))(conv1)

conv2=tf.keras.layers.ZeroPadding2D((1,1))(conv1)
conv2=tf.keras.layers.Convolution2D(128,3,3,activation='relu')(conv2)
conv2=tf.keras.layers.ZeroPadding2D((1,1))(conv2)
conv2=tf.keras.layers.Convolution2D(128,3,3,activation='relu')(conv2)
conv2=tf.keras.layers.MaxPooling2D((2,2),strides=(2,2))(conv2)

conv3=tf.keras.layers.ZeroPadding2D((1,1))(conv2)
conv3=tf.keras.layers.Convolution2D(256,3,3,activation='relu')(conv3)
conv3=tf.keras.layers.ZeroPadding2D((1,1))(conv3)
conv3=tf.keras.layers.Convolution2D(256,3,3,activation='relu')(conv3)
conv3=tf.keras.layers.ZeroPadding2D((1,1))(conv3)
conv3=tf.keras.layers.Convolution2D(256,3,3,activation='relu')(conv3)
conv3=tf.keras.layers.MaxPooling2D((2,2),strides=(2,2))(conv3)

conv4=tf.keras.layers.ZeroPadding2D((1,1))(conv2)
conv4=tf.keras.layers.Convolution2D(512,3,3,activation='relu')(conv4)
conv4=tf.keras.layers.ZeroPadding2D((1,1))(conv4)
conv4=tf.keras.layers.Convolution2D(512,3,3,activation='relu')(conv4)
conv4=tf.keras.layers.ZeroPadding2D((1,1))(conv4)
conv4=tf.keras.layers.Convolution2D(512,3,3,activation='relu')(conv4)
conv4=tf.keras.layers.MaxPooling2D((2,2),strides=(2,2))(conv4)

conv5=tf.keras.layers.ZeroPadding2D((1,1))(conv4)
conv5=tf.keras.layers.Convolution2D(512,3,3,activation='relu')(conv5)
conv5=tf.keras.layers.ZeroPadding2D((1,1))(conv5)
conv5=tf.keras.layers.Convolution2D(512,3,3,activation='relu')(conv5)
conv5=tf.keras.layers.ZeroPadding2D((1,1))(conv5)
conv5=tf.keras.layers.Convolution2D(512,3,3,activation='relu')(conv5)
conv5=tf.keras.layers.MaxPooling2D((2,2),strides=(2,2))(conv5)

conv6=tf.keras.layers.Flatten()(conv5)
conv6=tf.keras.layers.Dense(4096,activation='relu')(conv6)
conv6=tf.keras.layers.Dropout(0.5)(conv6)
conv6=tf.keras.layers.Dense(4096,activation='relu')(conv6)
conv6=tf.keras.layers.Dropout(0.5)(conv6)
outputs=tf.keras.layers.Dense(1000,activation='softmax')(conv6)

model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=8, epochs=1000)
model.save("FCNModel.h5")

In [ ]:
plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import random

test_id = random.choice(X_ids)
print(test_id)
img = tf.keras.preprocessing.image.load_img(f"/gdrive/MyDrive/ISBI_Dataset/train/{test_id}", target_size=(256, 256))
input_array = tf.keras.preprocessing.image.img_to_array(img)
input_array_model = np.array([input_array])
predictions = model.predict(input_array_model)
ax10=plt.subplot(121)
ax10.imshow(np.asarray(Image.open(f"/gdrive/MyDrive/ISBI_Dataset/train/{test_id}")))
ax11=plt.subplot(122)
ax11.imshow(tf.keras.preprocessing.image.array_to_img(np.squeeze(predictions)[:, :, np.newaxis]))
plt.show()

In [ ]:
ax12=plt.subplot(121)
ax12.imshow(np.asarray(tf.keras.preprocessing.image.array_to_img(np.squeeze(predictions)[:, :, np.newaxis])))
ax13=plt.subplot(122)
train_id=test_id.replace('volume','labels')
ax13.imshow(np.asarray(Image.open(f"/gdrive/MyDrive/ISBI_Dataset/label/{train_id}")))
plt.show()